<a href="https://colab.research.google.com/github/HuzaifaAliHPC/Chest_X_ray-/blob/main/Copy_of_Gradio_For_vgg_resnet_inceptionResnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Function 1
This function gets data from *HuggingFace*

In [ ]:
def get_data_huggingface():
  !pip install gradio
  !curl -Lo /content/archive.zip https://huggingface.co/datasets/HuzaifaHPC/chest_X_ray/resolve/main/archive.zip
  !curl -Lo /content/InceptionResnet_Chest_X_ray.h5 https://huggingface.co/HuzaifaHPC/INCRES_Chest_X_ray_3.h5/resolve/main/InceptionResnet_Chest_X_ray.h5
  !curl -Lo /content/VGG16_Chest_X_ray.h5 https://huggingface.co/HuzaifaHPC/VGG16_Chest_X_Ray/resolve/main/VGG16_Chest_X_ray.h5
  !curl -Lo /content/Resnet_Chest_X_ray.h5 https://huggingface.co/HuzaifaHPC/Resnet_Chest_X_Ray/resolve/main/Resnet_Chest_X_ray.h5
  !git clone https://github.com/HuzaifaAliHPC/Chest_X_ray-
  !unzip /content/archive.zip

get_data_huggingface()

# Import Dependencies

In [2]:
import cv2
import os
import re
import glob
import random
import warnings
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg
from platform import python_version
from PIL import Image
# from IPython.display import Image, display
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import vgg16, ResNet152V2, InceptionResNetV2
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, ImageDataGenerator
import gradio as gr
warnings.simplefilter(action='ignore',category=FutureWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
print(f'TensorFlow version: {tf.__version__}')
print(f'Python version: {python_version()}')

TensorFlow version: 2.13.0
Python version: 3.10.12


# Data Preprocessing

Create directories for training, testing and validation data

In [3]:
%matplotlib inline
BASE_PATH = "/content/chest_xray/"
TRAIN_PATH = BASE_PATH + "train/"
TEST_PATH = BASE_PATH + "test/"
VAL_PATH = BASE_PATH + "val/"

SHAPE = (224,224,3)
batch_size = 64
classes = ["NORMAL", "PNEUMONIA"]

print(f'train_path: {TRAIN_PATH}')
print(f'test_path: {TEST_PATH}')
print(f'val_path: {VAL_PATH}')

train_path: /content/chest_xray/train/
test_path: /content/chest_xray/test/
val_path: /content/chest_xray/val/


In [45]:
pneumonia_data_dir = TEST_PATH + classes[1]
# global variables
random_images = [os.path.join(pneumonia_data_dir, filename) for filename in random.sample([filename for filename in os.listdir(pneumonia_data_dir) if filename.endswith(".jpeg")], 4)]
sample_images = [
                  random_images

                 ]
classes = ["NORMAL", "PNEUMONIA"]

getting lengths of each directories

In [5]:
training_data = len(os.listdir(os.path.join(TRAIN_PATH, classes[0]))) + len(os.listdir(os.path.join(TRAIN_PATH, classes[1])))
test_data = len(os.listdir(os.path.join(TEST_PATH, classes[0]))) + len(os.listdir(os.path.join(TEST_PATH, classes[1])))
print(training_data, test_data)

5216 624


In [6]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                  featurewise_center=False,  # set input mean to 0 over the dataset
                                  samplewise_center=False,  # set each sample mean to 0
                                  featurewise_std_normalization=False,  # divide inputs by std of the dataset
                                  samplewise_std_normalization=False,  # divide each input by its std
                                  zca_whitening=False,  # apply ZCA whitening
                                  rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
                                  zoom_range = 0.2, # Randomly zoom image
                                  width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                                  height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                                  horizontal_flip = True,  # randomly flip images
                                  vertical_flip=False)  # randomly flip images)
test_datagen = ImageDataGenerator(rescale=1/255)
val_datagen = ImageDataGenerator(rescale=1/255)

In [7]:
train_generator = train_datagen.flow_from_directory(
                  TRAIN_PATH,
                  target_size = (SHAPE[0],SHAPE[1]),
                  batch_size = batch_size,
                  class_mode = 'categorical',
                  shuffle = True,
                  subset = None,
                  seed = 33
                  )

Found 5216 images belonging to 2 classes.


In [8]:
test_generator = test_datagen.flow_from_directory(
                TEST_PATH,
                target_size = (SHAPE[0],SHAPE[1]),
                batch_size = batch_size,
                class_mode = 'categorical',
                shuffle = True,
                subset = None,
                seed = 33
)

val_generator = val_datagen.flow_from_directory(
                VAL_PATH,
                target_size = (SHAPE[0],SHAPE[1]),
                batch_size = batch_size,
                class_mode = 'categorical',
                shuffle = True,
                subset = None,
                seed = 33
)

Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


Generate test labels for predictions, confusion matrix

In [9]:
test_num = test_generator.samples

label_test = []

for i in range((test_num // test_generator.batch_size)+1):
  X,y = test_generator.next()
  label_test.append(y)
label_test = np.argmax(np.vstack(label_test), axis=1)
label_test.shape

(624,)

# for VGG16

1. training
2. GradCAM



In [10]:
# This will run training scripts and save VGG16 model to /content/model.h5
def vgg_train():
  !python /content/Chest_X_ray-/vgg_training.py

In [11]:
def load_vgg():
  return tf.keras.models.load_model('/content/VGG16_Chest_X_ray.h5')

In [12]:

def VGG_test_accuracy():
  vgg_model = load_vgg()
  report_vgg = classification_report(label_test, np.argmax(vgg_model.predict(test_generator),axis=1))
  VGG_test_accuracy = float(report_vgg.split()[-2])
  return VGG_test_accuracy


In [13]:
def vgg_CM():
  vgg_model = load_vgg()
  y_preds_VGG16 = np.argmax(vgg_model.predict(test_generator),axis=1)

  cm_VGG16 = confusion_matrix(label_test, y_preds_VGG16)
  cmp_VGG16 = ConfusionMatrixDisplay(cm_VGG16, display_labels=classes)
  fig, ax = plt.subplots(figsize=(5,5))
  cmp_VGG16.plot(ax=ax)
  plt.xlabel('Predicted Class for VGG16')
  plt.ylabel('True Class')
  plt.savefig("pneumonia_confusion_matrix_for_VGG16.png", bbox_inches='tight')

heatmaps_pneumonia = []
img_path = "/content/chest_xray/test/PNEUMONIA/person100_bacteria_481.jpeg"

In [14]:
def pred_vgg(image):

    model = load_vgg()
    image = image.reshape((-1, 224, 224, 3))
    image = tf.keras.applications.vgg16.preprocess_input(image)
    prediction1 = model.predict(image).flatten()
    return {classes[i]: float(prediction1[i]) for i in range(2)}

In [15]:
def get_img_array(img_path, size):

    img = keras.utils.load_img(img_path, target_size=size)

    array = keras.utils.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 224, 224, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap_vgg16(img_array, last_conv_layer_name,  model = load_vgg(), pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions

    grad_model = keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)


    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

img_size = SHAPE
last_conv_layer_name_VGG16 = "block5_conv3"
preprocess_input_vgg16 = keras.applications.vgg16.preprocess_input
# Prepare image
# img_array = preprocess_input_vgg16(get_img_array(img_path, size=img_size))

from PIL import Image

def save_and_display_gradcam_vgg(img_path, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.utils.load_img(img_path)
    img = keras.utils.img_to_array(img)
    img_array = preprocess_input_vgg16(get_img_array(img_path, size=img_size))
    heatmap= make_gradcam_heatmap_vgg16(img_array, last_conv_layer_name_VGG16)
    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    pred = pred_vgg(get_img_array(img_path, size=img_size))

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.utils.array_to_img(superimposed_img)



    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    return Image.open(cam_path).resize((224,224)), pred

save_and_display_gradcam_vgg(img_path)

1/1 [==============================] - 1s 1s/step


<ipython-input-15-ae5e3178c947>:68: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jet = cm.get_cmap("jet")


(<PIL.Image.Image image mode=RGB size=224x224>,
 {'NORMAL': 0.9999352693557739, 'PNEUMONIA': 6.468516221502796e-05})

In [16]:
from PIL import Image
def CM_VGG16():
  vgg_CM()
  return Image.open("/content/pneumonia_confusion_matrix_for_VGG16.png")

In [17]:
def data_info_VGG16():
  # model_info = dict(training_data = training_data,
  #                   test_data = test_data,
  #                   test_accuracy = VGG_test_accuracy())
  return training_data, classes, test_data, VGG_test_accuracy()

In [18]:
def model_info_VGG16():
  model_name = "VGG16"
  model_size =  os.path.getsize('/content/VGG16_Chest_X_ray.h5') / (1024 * 1024)
  model_size = "{:.2f}".format(model_size) + " " + "MB"
  image_source = "imagenet"
  return model_name, model_size, image_source

# for Resnet

1. training
2. GradCAM

In [19]:
def resnet_train():
  !python /content/Chest_X_ray-/resnet_training.py

In [20]:
def load_res():
  return tf.keras.models.load_model('/content/Resnet_Chest_X_ray.h5')

In [21]:
def RESNET_test_accuracy():
  resnet_model = load_res()
  report_resnet = classification_report(label_test, np.argmax(resnet_model.predict(test_generator),axis=1))
  RESNET_test_accuracy = float(report_resnet.split()[-2])
  return RESNET_test_accuracy
# print(RESNET_test_accuracy)

In [22]:
def res_CM():
  resnet_model = load_res()
  y_preds_RESNET = np.argmax(resnet_model.predict(test_generator),axis=1)

  cm_RESNET = confusion_matrix(label_test, y_preds_RESNET)
  cmp_RESNET = ConfusionMatrixDisplay(cm_RESNET, display_labels=classes)
  fig, ax = plt.subplots(figsize=(5,5))
  cmp_RESNET.plot(ax=ax)
  plt.xlabel('Predicted Class for RESNET')
  plt.ylabel('True Class')
  plt.savefig("pneumonia_confusion_matrix_for_RESNET.png", bbox_inches='tight')


heatmaps_pneumonia = []
img_path = "/content/chest_xray/test/PNEUMONIA/person100_bacteria_481.jpeg"

In [23]:
def pred_res(image):

    model = load_res()
    image = image.reshape((-1, 224, 224, 3))
    image = tf.keras.applications.resnet_v2.preprocess_input(image)
    prediction1 = model.predict(image).flatten()
    return {classes[i]: float(prediction1[i]) for i in range(2)}

In [24]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.utils.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.utils.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap_resnet(img_array, last_conv_layer_name, model = load_res() , pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name_RESNET).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

img_size = SHAPE
last_conv_layer_name_RESNET = "conv5_block3_3_conv"
preprocess_input_RESNET = tf.keras.applications.resnet_v2.preprocess_input
# Prepare image
# img_array = preprocess_input_RESNET(get_img_array(img_path, size=img_size))



def save_and_display_gradcam_resnset(img_path, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.utils.load_img(img_path)
    img = keras.utils.img_to_array(img)
    img_array = preprocess_input_RESNET(get_img_array(img_path, size=img_size))
    heatmap = make_gradcam_heatmap_resnet(img_array, last_conv_layer_name_RESNET)
    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    pred = pred_res(get_img_array(img_path, size=img_size))

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.utils.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    return Image.open(cam_path).resize((224,224)), pred


# save_and_display_gradcam_resnset(img_path)

In [25]:
from PIL import Image
def CM_RESNET():
  res_CM()
  return Image.open("/content/pneumonia_confusion_matrix_for_RESNET.png")

In [26]:
def data_info_RESNET():
  # model_info = dict(training_data = training_data,
  #                   test_data = test_data,
  #                   test_accuracy = RESNET_test_accuracy())
  return training_data, classes, test_data, RESNET_test_accuracy()

In [27]:
def model_info_RESNET():
  model_name = "ResNet152V2"
  model_size =  os.path.getsize('/content/Resnet_Chest_X_ray.h5') / (1024 * 1024)
  model_size = "{:.2f}".format(model_size) + " " + "MB"
  image_source = "imagenet"
  return model_name, model_size, image_source

# InceptionResnet

1. training
2. GradCAM

In [28]:
def inceptionResnet_train():
  !python /content/Chest_X_ray-/inception_resnet_training.py

In [29]:
def load_incres():
  return tf.keras.models.load_model('/content/InceptionResnet_Chest_X_ray.h5')

In [30]:
def INCRES_test_accuracy():
  incres_model = load_incres()
  report_incres = classification_report(label_test, np.argmax(incres_model.predict(test_generator),axis=1))
  INCRES_test_accuracy = float(report_incres.split()[-2])
  return INCRES_test_accuracy
# print(INCRES_test_accuracy)

In [31]:
def incres_CM():
  incres_model = load_incres()
  y_preds_INCRES = np.argmax(incres_model.predict(test_generator),axis=1)

  cm_INCRES = confusion_matrix(label_test, y_preds_INCRES)
  cmp_INCRES = ConfusionMatrixDisplay(cm_INCRES, display_labels=classes)
  fig, ax = plt.subplots(figsize=(5,5))
  cmp_INCRES.plot(ax=ax)
  plt.xlabel('Predicted Class for INCRES')
  plt.ylabel('True Class')
  plt.savefig("pneumonia_confusion_matrix_for_INCRES.png", bbox_inches='tight')

heatmaps_pneumonia = []
img_path = "/content/chest_xray/test/PNEUMONIA/person100_bacteria_481.jpeg"

In [32]:
def pred_res(image):

    model = load_incres()
    image = image.reshape((-1, 224, 224, 3))
    image = tf.keras.applications.inception_resnet_v2.preprocess_input(image)
    prediction1 = model.predict(image).flatten()
    return {classes[i]: float(prediction1[i]) for i in range(2)}

In [33]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.utils.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.utils.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap_incres(img_array, last_conv_layer_name, model = load_incres(), pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name_INCRES).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

img_size = SHAPE
last_conv_layer_name_INCRES = "conv_7b"
preprocess_input_INCRES = tf.keras.applications.inception_resnet_v2.preprocess_input
# Prepare image
img_array = preprocess_input_INCRES(get_img_array(img_path, size=img_size))


def save_and_display_gradcam_incres(img_path, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.utils.load_img(img_path)
    img = keras.utils.img_to_array(img)
    img_array = preprocess_input_INCRES(get_img_array(img_path, size=img_size))
    heatmap = make_gradcam_heatmap_incres(img_array, last_conv_layer_name_INCRES)
    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    pred = pred_res(get_img_array(img_path, size=img_size))

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.utils.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    return Image.open(cam_path).resize((224,224)), pred


#save_and_display_gradcam_incres(img_path)

In [34]:
from PIL import Image
def CM_INCRES():
  incres_CM()
  return Image.open("/content/pneumonia_confusion_matrix_for_INCRES.png")

In [35]:
def data_info_INCRES():
  # model_info = dict(training_data = training_data,
  #                   test_data = test_data,
  #                   test_accuracy = INCRES_test_accuracy())
  return training_data, classes, test_data, INCRES_test_accuracy()

In [36]:
def model_info_INCRES():
  model_name = "InceptionResNetV2"
  model_size =  os.path.getsize('/content/InceptionResnet_Chest_X_ray.h5') / (1024 * 1024)
  model_size = "{:.2f}".format(model_size) + " " + "MB"

  image_source = "imagenet"
  return model_name, model_size, image_source

#Gradio UI

In [37]:
# gr.inputs.Image??

In [47]:
with gr.Blocks() as demo:
  with gr.Tab("VGG16"):
      with gr.Row():
          with gr.Column():
              training_data_vgg = gr.components.Textbox(label = "Training Data")
              classes_vgg = gr.components.Textbox(label = "Number of Classes")
              test_data_vgg = gr.components.Textbox(label = "Testing Data")
              test_accuracy_vgg = gr.components.Textbox(label = "Test Accuracy")
      btn = gr.Button("Get Data Info")
      btn.click(data_info_VGG16, outputs=[training_data_vgg, classes_vgg, test_data_vgg, test_accuracy_vgg])

      with gr.Row():
          with gr.Column():
              model_name_vgg = gr.components.Textbox(label = "Model Name")
              model_size_vgg = gr.components.Textbox(label = "Model Size")
              image_source_vgg = gr.components.Textbox(label = "Image Source")
      btn = gr.Button("Get Model Info")
      btn.click(model_info_VGG16, outputs=[model_name_vgg, model_size_vgg, image_source_vgg])


      with gr.Row():
          with gr.Column():
              out_vgg = gr.components.Textbox()
      btn = gr.Button("VGG retrain")
      btn.click(vgg_train, outputs=[out_vgg])

      with gr.Row():
          with gr.Column():
              out_cm_vgg = gr.components.Image(type='filepath')
      btn = gr.Button("Confusion Matrix")
      btn.click(CM_VGG16, outputs=[out_cm_vgg])

      with gr.Row():
          with gr.Column():
              seed_vgg = gr.components.Image(type='filepath')
          with gr.Column():
              out_vgg = gr.components.Image(type='filepath')
              label = gr.components.Label(num_top_classes = 2)

      btn = gr.Button("GradCAM")
      btn.click(save_and_display_gradcam_vgg, inputs=[seed_vgg], outputs=[out_vgg, label])
      gr.Examples(sample_images, inputs=[seed_vgg])

# for Resnet
  with gr.Tab("ResNet152V2"):
      with gr.Row():
          with gr.Column():
              training_data_res = gr.components.Textbox(label = "Training Data")
              classes_res = gr.components.Textbox(label = "Number of Classes")
              test_data_res = gr.components.Textbox(label = "Testing Data")
              test_accuracy_res = gr.components.Textbox(label = "Test Accuracy")
      btn = gr.Button("Get Data Info")
      btn.click(data_info_RESNET, outputs=[training_data_res, classes_res, test_data_res, test_accuracy_res])

      with gr.Row():
          with gr.Column():
              model_name_res = gr.components.Textbox(label = "Model Name")
              model_size_res = gr.components.Textbox(label = "Model Size")
              image_source_res = gr.components.Textbox(label = "Image Source")
      btn = gr.Button("Get Model Info")
      btn.click(model_info_RESNET, outputs=[model_name_res, model_size_res, image_source_res])


      with gr.Row():
          with gr.Column():
              out_res = gr.components.Textbox()
      btn = gr.Button("ResNet152V2 retrain")
      btn.click(resnet_train, outputs=[out_res])

      with gr.Row():
          with gr.Column():
              out_cm_res = gr.components.Image(type='filepath')
      btn = gr.Button("Confusion Matrix")
      btn.click(CM_RESNET, outputs=[out_cm_res])

      with gr.Row():
          with gr.Column():
              seed_res = gr.components.Image(type='filepath')
          with gr.Column():
              out_res = gr.components.Image(type='filepath')
              label = gr.components.Label(num_top_classes = 2)
      btn = gr.Button("GradCAM")
      btn.click(save_and_display_gradcam_resnset, inputs=[seed_res], outputs=[out_res, label])
      gr.Examples(sample_images, inputs=[seed_res])


# # For InceptionResnet
  with gr.Tab("InceptionResNetV2"):
      with gr.Row():
          with gr.Column():
              training_data_incres = gr.components.Textbox(label = "Training Data")
              classes_incres = gr.components.Textbox(label = "Number of Classes")
              test_data_incres = gr.components.Textbox(label = "Testing Data")
              test_accuracy_incres = gr.components.Textbox(label = "Test Accuracy")
      btn = gr.Button("Get Data Info")
      btn.click(data_info_INCRES, outputs=[training_data_incres, classes_incres, test_data_incres, test_accuracy_incres])

      with gr.Row():
          with gr.Column():
              model_name_incres = gr.components.Textbox(label = "Model Name")
              model_size_incres = gr.components.Textbox(label = "Model Size")
              image_source_incres = gr.components.Textbox(label = "Image Source")
      btn = gr.Button("Get Model Info")
      btn.click(model_info_INCRES, outputs=[model_name_incres, model_size_incres, image_source_incres])


      with gr.Row():
          with gr.Column():
              out_incres = gr.components.Textbox()
      btn = gr.Button("InceptionResNetV2 retrain")
      btn.click(inceptionResnet_train, outputs=[out_incres])

      with gr.Row():
          with gr.Column():
              out_cm_incres = gr.components.Image(type='filepath')
      btn = gr.Button("Confusion Matrix")
      btn.click(CM_INCRES, outputs=[out_cm_incres])

      with gr.Row():
          with gr.Column():
              seed_incres = gr.components.Image(type='filepath')
          with gr.Column():
              out_incres = gr.components.Image(type='filepath')
              label = gr.components.Label(num_top_classes = 2)
      btn = gr.Button("GradCAM")
      btn.click(save_and_display_gradcam_incres, inputs=[seed_incres], outputs=[out_incres, label])
      gr.Examples(sample_images, inputs=[seed_incres])

if __name__ == "__main__":
    demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://98237fe5fd1d922f6c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://98237fe5fd1d922f6c.gradio.live
